# Agent-based RAG system for Nerve API documentation

## Imports

In [1]:
from autogen.agentchat.contrib.retrieve_user_proxy_agent import RetrieveUserProxyAgent
from typing import Callable, List, Tuple, Union
import sys
import os

# Add path
sys.path.append('../..')

from inswitch.llm.model import get_openai_model_config
from inswitch.agent.basic import get_llm_agent, get_fixed_reply_agent
from inswitch.agent.ragagent import RagAgent
from inswitch.agent.filteragent import FilterAgent
from usecases.fill.filluc.mockupnerv.session import filter_task

flaml.automl is not available. Please install flaml[automl] to enable AutoML functionalities.


## Load all relevant Nerve API documents to be added to RAG

In [2]:
doc_1 = "https://docs.nerve.cloud/developer_guide/dna/"
doc_2 = "https://docs.nerve.cloud/developer_guide/networking/"
doc_3 = "https://docs.nerve.cloud/developer_guide/codesys/"
doc_4 = "https://docs.nerve.cloud/developer_guide/ms-api/api-pointers/"
doc_5 = "https://docs.nerve.cloud/developer_guide/ms-api/api-examples/"
doc_6 = "https://docs.nerve.cloud/developer_guide/ms-api/"

## Initialize agents

In [3]:
deploy_plan = """Deploy the following workloads for Machine: M00001 (Type: MTC)
      - Workload: ngix, Version: ngix_27 (1.27.2)
      - Workload: nodejs, Version: nodejs_23 (23.2.0)
      - Workload: alarm_record
      - Workload: mqtt_sender     
"""

# Moderator
moderator = get_fixed_reply_agent(
    name="moderator",
    reply = ""
)

# Input from plan announcer
deployment_plan_announcer = get_fixed_reply_agent(
    'deployment_plan_announcer',
    reply=deploy_plan
)

# Get input from plan announcer and filter the type of task
filter_agent = FilterAgent(
    name = "filter_agent",
    system_message = "You have access to a function/tool to classify what type of Nerve DNA-related task we need to do given an input.",
    max_internal_turns=2
)

filter_agent.register_api_function(
    filter_task,
    description = "This is the function to classify Nerve DNA-related tasks given an input. "
)

# Get type of task and fetch relevant documents
nerve_rag_agent = RagAgent(
    name = "rag_agent", 
    docs_path = [doc_1, doc_2, doc_3, doc_4, doc_5, doc_6],
    max_internal_turns=1
)

In [4]:
chat_result = moderator.initiate_chats(
    [
        {
            "recipient": deployment_plan_announcer,
            "message": "What should be deployed? on which machine?",
            "max_turns": 1,
            "summary_method": "last_msg",
        },
        {
            "recipient": filter_agent,
            "message": "Provide task",
            "max_turns": 1,
            "summary_method": "last_msg",
        },
        {
            "recipient": nerve_rag_agent,
            "message": "Provide documnents that are relevant to task",
            "max_turns": 1,
            "summary_method": "last_msg",
        }
    ]
)


********************************************************************************
Starting a new chat....

********************************************************************************
moderator (to deployment_plan_announcer):

What should be deployed? on which machine?

--------------------------------------------------------------------------------
deployment_plan_announcer (to moderator):

Deploy the following workloads for Machine: M00001 (Type: MTC)
      - Workload: ngix, Version: ngix_27 (1.27.2)
      - Workload: nodejs, Version: nodejs_23 (23.2.0)
      - Workload: alarm_record
      - Workload: mqtt_sender     


--------------------------------------------------------------------------------

********************************************************************************
Starting a new chat....

********************************************************************************
moderator (to filter_agent):

Provide task
Context: 
Deploy the following workloads for Machine: 

2024-12-06 09:12:26,904 - autogen.agentchat.contrib.retrieve_user_proxy_agent - INFO - Use the existing collection `autogen-docs`.


Trying to create collection.


max_tokens is too small to fit a single line of text. Breaking this line:
	 ...
Failed to split docs with must_break_at_empty_line being True, set to False.
2024-12-06 09:12:42,618 - autogen.agentchat.contrib.retrieve_user_proxy_agent - INFO - Found 54 chunks.


VectorDB returns doc_ids:  [['70aeedc4', 'f49ef914', '8daf7094', '1b3325fe', '3bc81191', 'dd57353c', '650aee3c', '2c5a8141', '0c7020d3', '31923c42']]
Adding content of doc 70aeedc4 to context.
Adding content of doc f49ef914 to context.
Adding content of doc 8daf7094 to context.
Adding content of doc 1b3325fe to context.
Adding content of doc 3bc81191 to context.
Adding content of doc dd57353c to context.
Adding content of doc 650aee3c to context.
rag_agent (to rag_agent_driver):

Retrieved Context: 
##### Getting the version ID

Depending on the type of workload, different calls need to be used to list workload versions and their IDs.

1. Expand the following call according to workload type.
   
   | Workload type | API call |
   | --- | --- |
   | **Docker Compose workloads** | GET /nerve/v3/workloads/{workloadID}/versions |
   | **Other workloads** | GET /nerve/v2/workloads/{workloadID} |
2. Select **Try it out**.
3. Enter the workload ID into the **Workload Id** field.
   
   ![Ente